# TODO: Come up with a title

Music has been an integral part of world cultures for thousands of years. Styles and songs have come and gone, but the concept of rating music has been a relatively new concept. Since the late 1950s, Billboard has been tracking the popularity of songs....

TODO: Finish this intro!

In [1]:
# Imports for the project

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
# Read in the data frame to start our analysis

smpl_df = pd.read_json('./full_data.json')

In [3]:
# Now, let's sort this data to get it formatted where each row is a full chart and each entry is a
# list of attributes relating to the song at that position

sorted_df = smpl_df.reindex_axis(sorted(smpl_df.columns), axis=1)

# Let's examine the dataframe
sorted_df.head()


,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,"[God's Plan, Drake, 1, 1, 4]","[Perfect, Ed Sheeran, 1, 2, 25]","[Finesse, Bruno Mars & Cardi B, 3, 3, 7]","[Havana, Camila Cabello Featuring Young Thug, ...","[Rockstar, Post Malone Featuring 21 Savage, 1,...","[Look Alive, BlocBoy JB Featuring Drake, 6, 0, 1]","[Meant To Be, Bebe Rexha & Florida Georgia Lin...","[New Rules, Dua Lipa, 6, 6, 29]","[All The Stars, Kendrick Lamar & SZA, 9, 31, 6]","[Stir Fry, Migos, 8, 8, 8]",...,"[Black Panther, Kendrick Lamar, 91, 0, 1]","[At The Club, Jacquees X Dej Loaf, 92, 92, 2]","[Codeine Dreaming, Kodak Black Featuring Lil W...","[The Long Way, Brett Eldredge, 93, 93, 2]","[Rock, Plies, 95, 99, 3]","[I Like Me Better, Lauv, 96, 0, 1]","[When We, Tank, 86, 0, 7]","[Criminal, Natti Natasha x Ozuna, 95, 95, 3]","[IDGAF, Dua Lipa, 88, 88, 5]","[Rewrite The Stars, Zac Efron & Zendaya, 70, 8..."
1,"[God's Plan, Drake, 1, 1, 3]","[Perfect, Ed Sheeran, 1, 2, 24]","[Finesse, Bruno Mars & Cardi B, 3, 3, 6]","[Havana, Camila Cabello Featuring Young Thug, ...","[Rockstar, Post Malone Featuring 21 Savage, 1,...","[New Rules, Dua Lipa, 6, 7, 28]","[Pray For Me, The Weeknd & Kendrick Lamar, 7, ...","[Stir Fry, Migos, 8, 12, 7]","[Meant To Be, Bebe Rexha & Florida Georgia Lin...","[Bad At Love, Halsey, 5, 6, 23]",...,"[Mayores, Becky G Featuring Bad Bunny, 74, 95,...","[At The Club, Jacquees X Dej Loaf, 92, 0, 1]","[The Long Way, Brett Eldredge, 93, 0, 1]","[Singles You Up, Jordan Davis, 94, 0, 1]","[Criminal, Natti Natasha x Ozuna, 95, 0, 2]","[Corazon, Maluma X Nego do Borel, 87, 100, 6]","[Sick Boy, The Chainsmokers, 65, 90, 3]","[A Girl Like You, Easton Corbin, 98, 0, 1]","[Rock, Plies, 99, 0, 2]","[Supplies, Justin Timberlake, 71, 0, 2]"
2,"[God's Plan, Drake, 1, 1, 2]","[Perfect, Ed Sheeran, 1, 2, 23]","[Finesse, Bruno Mars & Cardi B, 3, 5, 5]","[Havana, Camila Cabello Featuring Young Thug, ...","[Rockstar, Post Malone Featuring 21 Savage, 1,...","[Bad At Love, Halsey, 5, 6, 22]","[New Rules, Dua Lipa, 7, 8, 27]","[MotorSport, Migos, Nicki Minaj & Cardi B, 6, ...","[Say Something, Justin Timberlake Featuring Ch...","[Thunder, Imagine Dragons, 4, 9, 40]",...,"[Rubbin Off The Paint, YBN Nahmir, 46, 69, 13]","[El Farsante, Ozuna & Romeo Santos, 92, 0, 1]","[IDGAF, Dua Lipa, 92, 92, 3]","[Most People Are Good, Luke Bryan, 94, 0, 1]","[Mayores, Becky G Featuring Bad Bunny, 74, 94,...","[CC, Migos Featuring Gucci Mane, 96, 0, 1]","[Best Friend, Sofi Tukker Featuring NERVO, The...","[MIC Drop, BTS Featuring Desiigner, 28, 84, 10]","[King's Dead, Jay Rock, Kendrick Lamar, Future...","[Corazon, Maluma X Nego do Borel, 87, 93, 5]"
3,"[God's Plan, Drake, 1, 0, 1]","[Perfect, Ed Sheeran, 1, 2, 22]","[Havana, Camila Cabello Featuring Young Thug, ...","[Rockstar, Post Malone Featuring 21 Savage, 1,...","[Finesse, Bruno Mars & Cardi B, 3, 4, 4]","[Bad At Love, Halsey, 5, 5, 21]","[Diplomatic Immunity, Drake, 7, 0, 1]","[New Rules, Dua Lipa, 8, 8, 26]","[Thunder, Imagine Dragons, 4, 6, 39]","[No Limit, G-Eazy Featuring A$AP Rocky & Cardi...",...,"[Beautiful Trauma, P!nk, 91, 99, 5]","[IDGAF, Dua Lipa, 92, 92, 2]","[Corazon, Maluma X Nego do Borel, 87, 87, 4]","[Mayores, Becky G Featuring Bad Bunny, 74, 93,...","[The Greatest Show, Hugh Jackman, Keala Settle...","[Never Enough, Loren Allred, 88, 97, 4]","[My Dawg, Lil Baby, 71, 90, 7]","[I'll Name The Dogs, Blake Shelton, 56, 82, 20]","[Rock, Plies, 99, 0, 1]","[Female, Keith Urban, 69, 100, 5]"
4,"[Havana, Camila Cabello Featuring Young Thug, ...","[Perfect, Ed Sheeran, 1, 1, 21]","[Rockstar, Post Malone Featuring 21 Savage, 1,...","[Finesse, Bruno Mars & Cardi B, 3, 3, 3]","[Bad At Love, Halsey, 5, 7, 20]","[Thunder, Imagine Dragons, 4, 6, 38]","[No Limit, G-Eazy Featuring A$AP Rocky & Cardi...","[New Rules, Dua Lipa, 8, 11, 25]","[Too Good At Goodbyes, Sam Smith, 4, 8, 19]","[MotorSport, Migos, Nicki Minaj & Cardi B, 6, ...",...,"[King's Dead, Jay Rock, Kendrick Lamar, Future...","[IDGAF, Dua Lipa, 92

We have our song data! Now to extract and clean it up a bit

In [ ]:
# Indices:
# ChartWeek, SongName, Artist, PeakPos, LastPos, #Weeks

In [8]:
sorted_df[1][0][0] = 'God\'s Plan'